In [174]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 


Extraction des fichier csv 

In [175]:
train__set_features = pd.read_csv("training_set_features.csv")
test = pd.read_csv("test_set_features.csv")
train__set_features.shape

train__set_labels = pd.read_csv("training_set_labels.csv")

y_h1n1 = train__set_labels['h1n1_vaccine']
y_seasonal = train__set_labels['seasonal_vaccine']

train__set_features.head()

,respondent_id,h1n1_concern,h1n1_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,...,income_poverty,marital_status,rent_or_own,employment_status,hhs_geo_region,census_msa,household_adults,household_children,employment_industry,employment_occupation
0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,Below Poverty,Not Married,Own,Not in Labor Force,oxchjgsf,Non-MSA,0.0,0.0,NaN,NaN
1,1,3.0,2.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,...,Below Poverty,Not Married,Rent,Employed,bhuqouqj,"MSA, Not Principle City",0.0,0.0,pxcmvdjn,xgwztkwe
2,2,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,"<= $75,000, Above Poverty",Not Married,Own,Employed,qufhixun,"MSA, Not Principle City",2.0,0.0,rucpziij,xtkaffoo
3,3,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,...,Below Poverty,Not Married,Rent,Not in Labor Force,lrircsnp,"MSA, Principle City",0.0,0.0,NaN,NaN
4,4,2.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,...,"<= $75,000, Above Poverty",Married,Own,Employed,qufhixun,"MSA, Not Principle City",1.0,0.0,wxleyezf,emcorrxb


In [176]:
train = pd.concat([train__set_features, y_h1n1, y_seasonal], axis=1)


On va gerer les valeurs manquantes et les valeurs categoriels 

In [177]:
train.select_dtypes(include='object').isna().sum()


# la moitié de employement _industry et _occupation est manquante 
train['employment_occupation'].value_counts()

# essayer de creer catagorie big industrie // small industrie 
big_jobs = train['employment_occupation'].value_counts() > 1000
big_industries = train['employment_industry'].value_counts() > 1000

train['employment_occupation'] = train['employment_occupation'].isin(big_jobs[big_jobs == True].index)
train['employment_occupation'] = train['employment_occupation'].replace({False:0, True:1})

train['employment_industry'] = train['employment_industry'].isin(big_industries[big_industries == True].index)
train['employment_industry'] = train['employment_industry'].replace({False:0, True:1})



In [178]:
col = train.select_dtypes(exclude=(np.number)).columns

train = pd.get_dummies(train, columns=col)
test = pd.get_dummies(test, columns=col)



Creation des colonnes dans le cas ou la categorie n'était pas le test set 

In [179]:
for col in train.columns: 
    if not(col in test.columns):
        test[col] = pd.Series(np.zeros(test.shape[0]), name=col)


Estimation des valeurs manquantes grace aux KNN 


In [180]:

# essayer d'utiliser le test set pour les NA
 
from sklearn.neighbors import KNeighborsRegressor

def manage_na(X,na_col): 
    X = X.copy()
    numerical = X.select_dtypes(np.number)
    non_na_columns = numerical.loc[:,numerical.isna().sum() == 0].columns
    X_train = numerical.loc[numerical[na_col].isna() == False,non_na_columns]
    y_train = numerical.loc[numerical[na_col].isna() == False,na_col]
    X_na = numerical.loc[numerical[na_col].isna() == True, non_na_columns]
    knn = KNeighborsRegressor()
    knn.fit(X_train, y_train)
    y_na = knn.predict(X_na)
    X.loc[X[na_col].isna() == True, na_col] = y_na
    
    return X 

In [181]:
cond = train.corr()['seasonal_vaccine'].apply(abs).sort_values() > 0.2

nulls = train[cond.index[cond]].isna().sum() > 0


for col in nulls[nulls == True].index:
    train = manage_na(train, col)

nulls_test = test.isna().sum() > 0

for col in nulls_test[nulls_test == True].index:
    test = manage_na(test, col)


KeyError: 'employment_industry'

In [ ]:
nulls = train.isna().sum().sort_values() > 0
nulls.sort_values()


respondent_id                          False
census_msa_MSA, Not Principle  City    False
census_msa_MSA, Principle City         False
census_msa_Non-MSA                     False
education_Some College                 False
                                       ...  
behavioral_wash_hands                   True
behavioral_face_mask                    True
chronic_med_condition                   True
behavioral_touch_face                   True
doctor_recc_h1n1                        True
Length: 66, dtype: bool

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV, train_test_split


X = train


params = {
    "n_estimators": np.arange(10, 13,1), 
    "max_depth": np.arange(10, 13, 1),
    "min_samples_leaf": np.arange(9,12 , 1), 
}
grid = GridSearchCV(RandomForestRegressor(), params, cv=5)

nulls = X.isna().sum() > 0 

for col in nulls[nulls == True].index:
    X = manage_na(X, col)


X = X.drop(["h1n1_vaccine", "seasonal_vaccine"], axis=1)
test = test.drop(["h1n1_vaccine", "seasonal_vaccine"], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y_seasonal,  test_size=0.2, random_state=32,)

grid.fit(X_train, y_train)
grid.best_params_

KeyboardInterrupt: 

In [ ]:
rf = RandomForestRegressor(max_depth=13,min_samples_leaf=7, n_estimators=14)

rf.fit(X_train, y_train)

y_seasonal_predict = rf.predict(test)
y_seasonal

rf.score(X_test, y_test)
y_seasonal_predict.mean()
y_seasonal_predict

ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- employment_occupation_bxpfxfdn
- employment_occupation_ccgxvspp
- employment_occupation_cmhcxjea
- employment_occupation_dcjcmpih
- employment_occupation_dlvbwzss
- ...


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y_h1n1,  test_size=0.2, random_state=32,)

rf = RandomForestRegressor(max_depth=9,min_samples_leaf=9, n_estimators=11)

rf.fit(X_train, y_train)

grid.fit(X_train, y_train)
print(grid.best_params_)



y_h1n1_predict = rf.predict(test)


rf.score(X_test, y_test)
y_h1n1_predict.mean()

result = pd.concat([test["respondent_id"], pd.Series(y_h1n1_predict, name="h1n1_vaccine"), pd.Series(y_seasonal_predict, name="seasonal_vaccine")], axis=1)
result.to_csv("result_employment.csv", index=False)




In [ ]:
sub_format = pd.read_csv("submission_format.csv")


0    26707
1    26708
2    26709
3    26710
4    26711
Name: respondent_id, dtype: int64